#1) Instalar librerias y conexión al servidor.

In [ ]:
# Importing libraries
from google.colab import auth
from google.colab import files
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser  # Import dateutil.parser for automatic date parsing

# Formatting for viewing tables
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Authenticating Google Sheets
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#2) Descargar información & definir parametros.

In [ ]:
# Define the workbook and sheets
input_workbook_name = 'MPF - CSF'
output_workbook_name = 'CSFBBDD'

# Open the input workbook and get the list of sheets to process from the 'inputs' sheet
input_workbook = gc.open(input_workbook_name)
inputs_sheet = input_workbook.worksheet('Inputs')
sheets_to_process = inputs_sheet.get_values('F3:F7')
sheets_to_process = [sheet[0] for sheet in sheets_to_process if sheet]

print("Sheets to process:", sheets_to_process)  # Debug print

# Get the last real value date from cell C13
last_real_value_date_str = inputs_sheet.cell(13, 3).value
last_real_value_date = parser.parse(last_real_value_date_str)
last_real_value_date_str2 = inputs_sheet.cell(14, 3).value
last_real_value_date2 = parser.parse(last_real_value_date_str2)

Sheets to process: ['Productividad', 'Ingresos', 'Tarifas', 'Costos', 'Gastos']


#3) Consolidación Base BBDD + Formato.

In [ ]:
# Initialize an empty DataFrame for consolidation
consolidated_df = pd.DataFrame(columns=[
    "Category", "Level1", "Level2", "Level3", "Level4", "Level5", "Level6", "Level7", "Level8", "Date", "Value"
])

# Function to check if a string can be parsed as a date
def is_valid_date(date_str):
    try:
        parser.parse(date_str)
        return True
    except (parser.ParserError, TypeError):
        return False

# Process each sheet
for sheet_name in sheets_to_process:
    print("Processing sheet:", sheet_name)
    sheet = input_workbook.worksheet(sheet_name)
    data = sheet.get_all_values()
    df = pd.DataFrame(data[3:], columns=data[2])  # Data starting from row 4, columns from row 3
    category = data[1][3]  # D2 value

    # Iterate through rows in the sheet
    for i, row in df.iterrows():
        if row[0]:  # Check if Level 1 (column A) is not empty
            # Iterate through columns starting from column K (index 26)
            for col_index in range(13, len(row)):
                if row[col_index]:
                    # Clean and convert value to float if possible
                    value_str = row[col_index].replace(',', '')
                    if '%' in value_str:
                        value = float(value_str.replace('%', '')) / 100
                    else:
                        try:
                            value = float(value_str)
                        except ValueError:
                            value = None  # Set to None if not a valid number
                    if isinstance(value, float):
                        value = round(value, 2)

                    # Parse date using dateutil.parser
                    date_str = data[2][col_index]
                    if is_valid_date(date_str):
                        date = parser.parse(date_str)  # Use dateutil.parser to parse the date

                        # Create a new row in the DataFrame
                        consolidated_row = pd.Series([
                            category,
                            row[0],  # Level 1
                            row[1],  # Level 2
                            row[2],  # Level 3
                            row[3],  # Level 4
                            row[4],  # Level 5
                            row[5],  # Level 6
                            row[6],  # Level 7
                            row[7],  # Level 8
                            date,   # Date
                            value   # Value
                        ], index=consolidated_df.columns)

                        # Append the row to the consolidated DataFrame
                        consolidated_df = pd.concat([consolidated_df, consolidated_row.to_frame().T], ignore_index=True)



# Convert 'Date' column to datetime
consolidated_df['Date'] = pd.to_datetime(consolidated_df['Date'])

# Ensure 'Value' column is numeric, replacing non-numeric values with NaN
consolidated_df['Value'] = pd.to_numeric(consolidated_df['Value'], errors='coerce')

# Sort values by Category, Level1, Level2, and Date for easier processing
consolidated_df.sort_values(by=['Category', 'Level1', 'Level2', 'Date'], inplace=True)

consolidated_df = consolidated_df.fillna(0)


consolidated_df = consolidated_df.pivot_table(index=['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5', 'Level7', 'Level8', 'Date'], columns='Level6', values='Value', aggfunc='sum').reset_index()
consolidated_df['Real_o_Proyeccion'] = np.where(
    consolidated_df['Date'] <= last_real_value_date,
    consolidated_df['Real'],
    consolidated_df['Proyec.']
)

Processing sheet: Productividad


Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-7-72b2d14172e4>:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[4],  # Level 5
<ipython-input-7-72b2d14172e4>:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[5],  # Level 6
<ipython-input-7-72b2d14172e4>:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[6],  # Level 7
<ipython-input-7-72b2d14172e4>:55: FutureWarning: Series.__getitem__ treati

Processing sheet: Ingresos


Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-7-72b2d14172e4>:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[6],  # Level 7
<ipython-input-7-72b2d14172e4>:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[7],  # Level 8
<ipython-input-7-72b2d14172e4>:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[col_index]:
<ipython-input-7-72b2d14172e4>:29: FutureWarning: Series.__getitem__ treati

Processing sheet: Tarifas


Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-7-72b2d14172e4>:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[6],  # Level 7
<ipython-input-7-72b2d14172e4>:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[7],  # Level 8
<ipython-input-7-72b2d14172e4>:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[col_index]:
<ipython-input-7-72b2d14172e4>:29: FutureWarning: Series.__getitem__ treati

Processing sheet: Costos


Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-7-72b2d14172e4>:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0],  # Level 1
<ipython-input-7-72b2d14172e4>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[1],  # Level 2
<ipython-input-7-72b2d14172e4>:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[2],  # Level 3
<ipython-input-7-72b2d14172e4>:51: FutureWarning: Series.__getitem__ treati

Processing sheet: Gastos


Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-7-72b2d14172e4>:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_str = row[col_index].replace(',', '')
<ipython-input-7-72b2d14172e4>:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0],  # Level 1
<ipython-input-7-72b2d14172e4>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[1],  # Level 2
<ipython-input-7-72b2d14172e4>:50: FutureWarning: 

#4) Calculos valores adicionales.

* Real_Acum_Año
* Real_o_Proyec_Acum_Año
* Ppto_Acum_Año
* Real_Ant
* Real_o_Proyec_Ant

In [ ]:
# Add columns for additional calculations
# Calculate accumulated values for the year, previous month's value, and accumulated value for the same period last year
consolidated_df['Real_Acum_Año'] = consolidated_df.groupby(['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5', 'Level7', 'Level8', consolidated_df['Date'].dt.year])['Real'].cumsum()
consolidated_df['Real_o_Proyec_Acum_Año'] = consolidated_df.groupby(['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5', 'Level7', 'Level8', consolidated_df['Date'].dt.year])['Real_o_Proyeccion'].cumsum()
consolidated_df['Ppto_Acum_Año'] = consolidated_df.groupby(['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5', 'Level7', 'Level8', consolidated_df['Date'].dt.year])['Ppto.'].cumsum()

#Meses anteriores
consolidated_df['Real_Ant'] = consolidated_df.groupby(['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5', 'Level7', 'Level8'])['Real'].shift(1)
consolidated_df['Real_o_Proyec_Ant'] = consolidated_df.groupby(['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5', 'Level7', 'Level8'])['Real_o_Proyeccion'].shift(1)


#5) Merge valores Last_Year.

In [ ]:
# Create a shifted DataFrame for the previous year accumulated values
consolidated_df['Year'] = consolidated_df['Date'].dt.year
consolidated_df['Month'] = consolidated_df['Date'].dt.month


last_year_df = consolidated_df.copy()
last_year_df['Year'] += 1

# Merge to get last year's accumulated values
consolidated_df = consolidated_df.merge(
    last_year_df[['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5',  'Level7', 'Level8', 'Month', 'Year', 'Real_Acum_Año','Real_o_Proyec_Acum_Año','Real_o_Proyeccion']],
    left_on=['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5',  'Level7', 'Level8', 'Month', 'Year'],
    right_on=['Category', 'Level1', 'Level2', 'Level3', 'Level4', 'Level5',  'Level7', 'Level8', 'Month', 'Year'],
    suffixes=('', '_Last_Year'),
    how='left'
)

consolidated_df['Real_o_Proyeccion_12M'] = consolidated_df['Real_o_Proyeccion_Last_Year']
consolidated_df['Real_Acum_Last_Year'] = consolidated_df['Real_Acum_Año_Last_Year']
consolidated_df['Real_o_Proyec_Acum_Last_Year'] = consolidated_df['Real_o_Proyec_Acum_Año_Last_Year']
consolidated_df.drop(columns=['Real_Acum_Año_Last_Year','Real_o_Proyec_Acum_Año_Last_Year' ,'Real_o_Proyeccion_Last_Year','Year', 'Month'], inplace=True)

#6) Calculo metricas.

* MoM_Abs
* MoM_Porc.
* YoY_Porc.
* Cump_Ppto_Abs
* Cump_Ppto_Porc
* Cump_Ppto_Abs_Acum
* Cump_Ppto_Porc_Acum

In [ ]:
#Diferencias
consolidated_df['MoM_Abs'] = round(consolidated_df['Real_o_Proyeccion'] - consolidated_df['Real_o_Proyec_Ant'],2)
consolidated_df['MoM_Porc.'] = round((consolidated_df['MoM_Abs'] / consolidated_df['Real_o_Proyec_Ant']) * 100,2)

consolidated_df['YoY_Abs'] = round(consolidated_df['Real_o_Proyec_Acum_Año'] - consolidated_df['Real_o_Proyec_Acum_Last_Year'],2)
consolidated_df['YoY_Porc.'] = round((consolidated_df['YoY_Abs'] / consolidated_df['Real_o_Proyec_Acum_Last_Year']) * 100,2)

consolidated_df['Cump_Ppto_Abs'] = round(consolidated_df['Real_o_Proyeccion'] - consolidated_df['Ppto.'],2)
consolidated_df['Cump_Ppto_Porc.'] = np.where(
    consolidated_df['Ppto.'] < 0,
    round((1 - (consolidated_df['Real_o_Proyeccion'] / consolidated_df['Ppto.'] - 1))*100, 2),
    round((consolidated_df['Real_o_Proyeccion'] / consolidated_df['Ppto.']) * 100, 2)
)


consolidated_df['Cump_Ppto_Abs_Acum'] = round(consolidated_df['Real_o_Proyec_Acum_Año'] - consolidated_df['Ppto_Acum_Año'],2)
consolidated_df['Cump_Ppto_Porc_Acum'] = np.where(
    consolidated_df['Ppto.'] < 0,
    round((1 - (consolidated_df['Real_o_Proyec_Acum_Año'] / consolidated_df['Ppto_Acum_Año'] - 1))*100, 2),
    round((consolidated_df['Real_o_Proyec_Acum_Año'] / consolidated_df['Ppto_Acum_Año']) * 100, 2)
)

#7) Formato JSON + Actualizar hojas de cálculo en Google Sheets.

In [ ]:
consolidated_df=consolidated_df.fillna(0)

# Replace non-JSON-compliant float values with None
consolidated_df.replace([float('inf'), float('-inf'), float('nan')], None, inplace=True)

# Convert 'Date' column to string to avoid JSON serialization issues
consolidated_df['Date'] = consolidated_df['Date'].dt.strftime('%Y-%m-%d')

#Crea marca de ultima fecha real
consolidated_df['Marca_Fecha'] = ((consolidated_df['Date'] == last_real_value_date.strftime('%Y-%m-%d'))).astype(int)
consolidated_df['Marca_Fecha2'] = ((consolidated_df['Date'] == last_real_value_date2.strftime('%Y-%m-%d'))).astype(int)

# Create a new workbook and write the consolidated data
output_sheet = gc.open(output_workbook_name).worksheet('BBDD')
output_sheet.clear()
output_sheet.update([consolidated_df.columns.values.tolist()] + consolidated_df.fillna('').values.tolist())

print('Proceso_Terminado')

Proceso_Terminado
